In [1]:
import config

Available soynlp == 0.0.492
Available politician_news_analyzer == 0.0.1
Available politician_news_dataset


politician_news_dataset 의 News 는 각 카테고리의 뉴스를 일별로 로딩하는 기능을 제공합니다. News.num_docs 에는 날짜 별 뉴스 기사가 저장되어 있습니다. News class 의 더 자세한 사용법은 https://github.com/lovit/politician_news_dataset 의 README 를 확인하세요.

DailyStream 은 하루에 발생한 모든 문서를 하나의 문서를 병합한 class 입니다.

In [2]:
from politician_news_dataset import News
from soynlp.normalizer import only_hangle_number

class DailyStream:
    """
    A doc for a date
    """

    def __init__(self, news, preprocessing=None, verbose=False):
        self.news = news
        if preprocessing is None:
            preprocessing = only_hangle_number
        self.preprocessing = preprocessing
        self.verbose = verbose

    def __iter__(self):
        n_dates = len(self.news.num_docs)
        for i, (date, _) in enumerate(self.news.num_docs):
            docs = ' '.join(self.news.get_news(begin_date=date, end_date=date))
            docs = only_hangle_number(docs)
            yield docs
            if self.verbose:
                print('\r{} / {} dates ... '.format(i+1, n_dates), end='')
        print('\rterminated iteration for {0} / {0} dates '.format(n_dates))

news = News(category=0) # '김무성' 뉴스
num_dates = len(news.num_docs)
print('{} - {}: {} dates'.format(news.num_docs[0][0], news.num_docs[-1][0], num_dates))

idx_to_date = [date for date, _ in news.num_docs]
date_to_idx = {date:idx for idx, date in enumerate(idx_to_date)}

2013-01-01 - 2019-03-10: 2224 dates


NounLMatchTokenizer 는 어절의 L part 에 존재하는 가장 긴 명사만을 선택하는 토크나이저 입니다. 연속된 두 개의 명사는 복합명사로 취급합니다.

In [3]:
from soynlp.tokenizer import NounLMatchTokenizer

noun_tokenizer = NounLMatchTokenizer(nouns = {'명사', '집합', '사전', '토크나이저'})
noun_tokenizer.tokenize('이 토크나이저는 명사집합 사전을 이용하여 어절의 왼쪽에서 명사만 선택합니다')

['토크나이저', '명사집합', '사전', '명사']

2013-01-01 - 2019-03-10 (2,224 일) 간의 뉴스 집합에서 명사를 추출한 뒤, 이를 이용하여 토크나이저를 만듭니다. 그 뒤, 이를 이용하여 일자 별 Bag-of-Words Model 을 학습합니다.

이 작업도 약 10 분의 계산 시간과 5GB 에 가까운 메모리가 필요합니다.

In [4]:
from soynlp.noun import LRNounExtractor_v2
from sklearn.feature_extraction.text import CountVectorizer

# train noun extractor
daily_stream = DailyStream(news, verbose=True)
noun_extractor = LRNounExtractor_v2()
noun_scores = noun_extractor.train_extract(daily_stream, min_noun_frequency=10)
noun_scores = {n:s for n,s in noun_scores.items() if len(n) > 1}

# build tokenizer
noun_tokenizer = NounLMatchTokenizer(noun_scores)

# vectorize
vectorizer = CountVectorizer(tokenizer=noun_tokenizer.tokenize)
x = vectorizer.fit_transform(daily_stream)
vocab_to_idx = vectorizer.vocabulary_
idx_to_vocab = [vocab for vocab, idx in sorted(vocab_to_idx.items(), key=lambda x:x[1])]
x.shape

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
terminated iteration for 2224 / 2224 dates 
[EojeolCounter] n eojeol = 1435928 from 2224 sents. mem=0.355 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=57572836, mem=2.968 Gb
[Noun Extractor] batch prediction was completed for 383950 words
[Noun Extractor] checked compounds. discovered 341050 compounds
[Noun Extractor] postprocessing detaching_features : 65307 -> 63066
[Noun Extractor] postprocessing ignore_features : 63066 -> 62668
[Noun Extractor] postprocessing ignore_NJ : 62668 -> 61279
[Noun Extractor] 61279 nouns (341050 compounds) with min frequency=10
[Noun Extractor] flushing was done. mem=3.530 Gb                    
[Noun Extractor] 78.66 % eojeols are covered
terminated iteration for 2224 / 2224 dates 


(2224, 227577)

politician_news_analyzer 안에는 단어 등장 빈도를 이용하는 키워드 추출 함수와 KR-WordRank 의 핵심 문장 추출 함수가 구현되어 있습니다. 이를 이용하여 begin_date 부터 end_date 기간의 뉴스를 요약하는 함수를 만듭니다.

margin 은 begin_date 보다 margin 일 만큼 이전의, end_date 보다 margin 일 만큼 이후의 뉴스를 reference documents 로 이용한다는 의미입니다.

In [9]:
import warnings
warnings.filterwarnings('ignore')

from politician_news_analyzer.summarizer import proportion_keyword
from politician_news_analyzer.summarizer import diverse_keysentences

def summarize(begin_date, end_date, margin=15, topk1=100, topk2=30, num_sents=5):
    n_dates = len(idx_to_date)

    def find_idx(date):
        for i, date_i in enumerate(idx_to_date[:-1]):
            if (date_i <= date < idx_to_date[i+1]):
                return i
        return n_dates - 1

    bdate = find_idx(begin_date)
    edate = find_idx(end_date)
    pos_idx = [d for d in range(bdate, edate + 1)]
    ref_idx = [d for d in range(max(0, bdate - margin), bdate)]
    ref_idx += [d for d in range(edate + 1, min(edate + 1 + margin, n_dates))]

    keywords = proportion_keyword(x, pos_idx, idx_to_vocab, ref_idx, topk1, topk2)
    keyword_score = {w:s for w,s,_ in keywords}

    docs = news.get_news(begin_date, end_date)
    raw_sents = [sent for doc in docs for sent in doc.split('  ')]
    sents = [only_hangle_number(sent) for sent in raw_sents]

    keysents = diverse_keysentences(keyword_score, sents, num_sents, raw_texts=raw_sents, diversity=0.5)
    return keywords, keysents

def print_summary(begin_date, end_date, keywords, keysents):
    keywords_ = ', '.join(word for word, _, _ in keywords)
    print('{} - {}\n'.format(begin_date, end_date))
    print('[Keywords]\n{}\n'.format(keywords_))
    print('[Key-sentences]')
    for sent in keysents:
        print(' - {}'.format(sent))

2017-05-24 에는 '노룩패스' 사건이 있었고, 다음날부터 이 사건에 대한 기사가 주를 이뤘습니다. 이와 관련된 키워드와 핵심 문장이 선택되었습니다.

In [10]:
begin_date = '2017-05-25'
end_date = '2017-05-26'

keywords, keysents = summarize(begin_date, end_date)
print_summary(begin_date, end_date, keywords, keysents)

2017-05-25 - 2017-05-26

[Keywords]
정관용, 정두언, 유병재, 김현정, 패러디, 25일, 공항, 노룩패스, 소개, 영상, 화제, 패스, 정유라, 공개, 캐리어, 24일, 정치인, 1년, 행동, 이들, 입국, 반납, 세비, 수행원, 장면, 한국, 사진, 이야기, 이낙연, 자신

[Key-sentences]
 - 농구 경기에서 상대편 수비수를 속이기 위해 다른 방향을 보면서 공을 넘기는 동작. ‘노룩패스’(No Look Pass)다. 바른정당의 김무성 의원이 어제 온종일 ‘노룩패스’의 주인공이 돼 인터넷을 후끈 달궜다. 일본 여행에서 돌아온 김 의원이 공항에서 찍힌 몇 초간의 동영상 때문이다. 김 의원은 입국장에 들어서면서 수행비서를 쳐다보지도 않고 걷는 속도를 유지한 채 캐리어를 한 손으로 휙 밀어 정확히 전달에 성공(?)했다. 화면 안으로 잽싸게 뛰어든 수행원이 깔끔하게 캐리어를 받는 장면은 사전 연습이나 한 듯 익숙했다.덕분에 아침 일찍부터 김 의원은 실시간 검색어 선두를 달렸다. 뒤를 안 보고도 공을 뒤로 패스하는 ‘비하인드백패스’(Behind Back Pass)까지 덩달아 인기 검색어로 떴다. 해외 온라인 커뮤니티들에서도 화제였다. 구글, 페이스북, 트위터에 이은 인기 커뮤니티 레딧은 ‘한국 정치인의 스웨그(Swag?허세)’라고 꼬집었다. 어느 온라인 쇼핑몰은 문제의 캐리어를 ‘노룩패스 가방’이라며 홍보했다.
 - 공개된 영상에는 유병재가 공항 문을 나오면서 스태프로 보이는 사람에게 자신의 캐리어를 굴려 전달하는 모습이 담겨 있다. 김 의원의 ‘캐리어 노 룩 패스’를 패러디한 것으로 해당 영상은 25일 오전 11시 현재, 17만 명이 넘는 네티즌들이 페이스북 ‘공감’을 누르는 등 큰 관심을 끌고 있다.
 - 방송인 유병재의 ‘노룩패스’ 패러디 영상이 화제를 모으고 있다.
 - 지미 펄론은 지난 24일(현지시각) 방송된 미국 NBC ‘지미 팰런의 투나잇 쇼’(The Tonight Show Starring Jimmy Fallon)

2018-06-15 ~ 2018-06-22 에는 자유한국당 의원 간의 총선 출마에 관련된 논쟁들이 있던 시기입니다. 이들에 대한 키워드와 핵심 문장이 선택되었습니다.

In [11]:
begin_date = '2018-06-15'
end_date = '2018-06-22'

keywords, keysents = summarize(begin_date, end_date)
print_summary(begin_date, end_date, keywords, keysents)

2018-06-15 - 2018-06-22

[Keywords]
혁신안, 중앙당, 해체, 선언, 메모, 불출마, 중진, 초선, 혁신, 참패, 잘못, 정당, 쇄신, 의원총회, 국민들, 탄핵, 앞으로, 이번, 논의, 보수, 총선, 의총, 상당, 당의, 갈등, 지적, 바른미래당, 대해서, 어떤, 권한대행

[Key-sentences]
 - 한국당은 15일 오후 국회에서 비상 의원총회를 열어 새 지도부 선출을 비롯한 당의 진로를 논의했다. 김성태 원내대표는 이날 공개발언에서 “우리 당이 처한 정치생태계를 바꿔야 새로운 세력이 등장하고 새로운 도전도 가능해진다”며 “물러날 분들은 뒤로 물러나고 확실한 세대교체를 이뤄야 한다”고 강조했다. 한국당의 상황은 그러나 김 원내대표의 말과는 반대로 가고 있다. 지적만 난무할 뿐 ‘내가 잘못했으니···’라는 인정과 책임은 찾아보기 어렵다. 이번 사태를 1인칭에서 바라봐야 할 당사자들이 3인칭 관찰자 시점에 머물러 있는 셈이다. 실제로 차기 당권 주자로 거론되는 중진들은 잇따라 반성문을 쏟아냈다. 5선의 심재철 의원은 페이스북에 “남은 것은 통렬한 자기반성과 철저한 자기혁신밖에 없다”고 주장했고 4선의 정우택 의원은 “보수는 죽었다. 무엇을 바꿔야 하는지 돌이켜보고 가슴에 새겨 실천하겠다”고 밝혔다. 4선인 나경원 의원 역시 “당과 보수가 잘못된 길을 가는데도 더 용기 있게 말하지 못한 것을 반성한다”며 “모두 버리고 새로 시작해야 한다”고 강조했다. 초선 의원들은 중진들이 당면과제에 뒷짐 지고 있다며 공개 비판에 나섰다. 김순례·김성태(비례)·성일종·이은권·정종섭 의원은 기자회견을 열어 “10년간 보수정치의 실패에 책임이 있는 중진들은 정계 은퇴를 하고 당을 제대로 이끌지 못한 중진은 당 운영의 전면에 나서지 말라”고 요구했다. 특정 의원을 지칭하지는 않았지만 계파싸움에 골몰하며 내홍을 부추겨온 중진을 싸잡아 비판한 것으로 보인다. 이 같은 내분을 두고 정치권에서는 “한국당 의원들이 ‘잘못도 했고 희생도 해야 하는데 그게 나는 아

물론 이 방법이 news stream 을 제대로 요약하는 방법은 아닙니다만, 큰 흐름을 설명할 수 있는 키워드와 핵심 문장은 위의 간단한 코드로 추출할 수 있습니다. 또한 politician news dataset 을 이용하는 연습을 할 수 있습니다.